In [1]:
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime, timedelta
import json

In [5]:
CHARLSON_CONDITIONS = {
    'mi': {
        '9': ["410", "412"],
        '10': ["I21", "I22", "I25.2"]
    },
    'chf': {
        '9': ["398.91", "402.01","402.11","402.91","404.01","404.03","404.11","404.13","404.91","404.93","425.4","425.5","425.6","425.7","425.8","425.9","428"],
        '10': ["I09.9", "I11.0", "I13.0", "I13.2", "I25.5", "I42.0", "I42.5", "I42.6", "I42.7", "I42.8", "I42.9", "I43", "I50", "P29.0"]
    },
    'pvd': {
        '9': ["093.0", "440","441","443.1","443.2","443.3","443.4","443.5","443.6","443.7","443.8","443.9","557.1","557.9","V43.4"],
        '10': ["I70", "I71", "I73.1","I73.8","I73.9","I77.1","I79.0","I79.2","K55.1","K55.8","K55.9","Z95.8","Z95.9"]
    },
    'cevd': {
        '9': ["362.34", "430","431","432","433","434","435","436","437","438"],
        '10': ["G45", "G46", "H34.0","I60", "I61", "I62","I63","I64","I65","I66","I67","I68","I69"]
    },
    'dementia': {
        '9': ["290","294.1","331.2"],
        '10': ["F00","F01","F02","F03","F05.1","G30","G31.1"]
    },
    'copd': {
        '9': ["416.8","416.9","490", "491","492","493","494","495","496","497","498","499","500","501","502","503","504","505","506.4","508.1", "508.8"],
        '10': ["I27.8","I27.9","J40", "J41", "J42","J43","J44","J45","J46","J47","J60","J61","J62","J63","J64","J65","J66","J67","J68.4","J70.1", "J70.3"]
    },
    'rheumd': {
        '9': ["446.5", "710.0", "710.1", "710.2", "710.3", "710.4", "714.0", "714.1", "714.2", "714.8", "725"],
        '10': ["M05", "M06", "M31.5", "M32", "M33", "M34", "M35.1", "M35.3", "M36.0"]
    },
    'pud': {
        '9': ["531", "532", "533", "534"],
        '10': ["K25", "K26", "K27", "K28"]
    },
    'mld': {
        '9': ["070.22", "070.23", "070.32", "070.33", "070.44", "070.54", "070.6", "070.9", "570", "571", "573.3", "573.4", "573.8", "573.9", "V42.7"],
        '10': ["B18", "K70.0", "K70.1", "K70.2", "K70.3", "K70.9", "K71.3", "K71.4", "K71.5", "K71.7", "K73", "K74", "K76.0", "K76.2", "K76.3", "K76.4", "K76.8", "K76.9", "Z94.4"]
    },
    'msld': {
        '9': ["456.0", "456.1", "456.2", "572.2", "572.3", "572.4","572.5","572.6","572.7", "572.8"],
        '10': ["I85.0","I85.9", "I86.4","I98.2", "K70.4", "K71.1", "K72.1", "K72.9", "K76.5", "K76.6", "K76.7"]
    },
    'diab': {
        '9': ["250.0", "250.1", "250.2", "250.3", "250.8", "250.9"],
        '10': ["E10.0", "E10.1", "E10.6", "E10.8","E10.9","E11.0","E11.1","E11.6","E11.8","E11.9","E13.0","E13.1","E13.6","E13.8","E13.9"]
    },
    'dia_w_c': {
        '9': ["250.4", "250.5", "250.6", "250.7"],
        '10': ["E10.2", "E10.3", "E10.4", "E10.5", "E10.7", "E11.2", "E11.3", "E11.4", "E11.5", "E11.7", "E13.2", "E13.3", "E13.4", "E13.5", "E13.7"]
    },
    'hp': {
        '9': ["334.1", "342", "343", "344.0","344.1","344.2","344.3","344.4","344.5","344.6","344.9"],
        '10': ["G04.1", "G11.4", "G80.1", "G80.2", "G81", "G82", "G83.0","G83.1","G83.2","G83.3","G83.4","G83.9"]
    },
    'mrend': {
        '9': ["403.00","403.10", "403.90", "404.00", "404.01", "404.10", "404.11", "404.90", "404.91", "584", "585.6", "589"],
        '10': ["I12.9","I13.0","I13.10","N03", "N05","N18.1", "N18.2","N18.3","N18.4","N18.9", "Z49.0"]
    },
    'srend': {
        '9': ["403.01", "403.11", "403.91", "404.02", "404.03", "404.12", "404.13", "404.92", "404.93","582","583.0","583.1","583.2","583.3","583.4","583.5","583.6","583.7","585.5", "585.6","586","588.0","V42.0","V45.1","V56"],
        '10': ["I12.0", "I13.11","I13.2","N18.5","N18.6","N19", "N25.0","Z49","Z94.0", "Z99.2"]
    },
    'aids': {
        '9': ["112", "180", "114", "117.5", "007.4", "078.5", "348.3", "054", "115", "007.2", "176", "200", "201", "202", "203", "204", "205", "206", "207", "208", "209", "031", "010", "011", "012", "013", "014", "015", "016", "017", "018", "136.3", "V12.61", "046.3", "003.1", "130", "799.4"],
        '10': ["B37", "C53", "B38", "B45", "A07.2", "B25", "G93.4", "B00", "B39", "A07.3", "C46", "C81", "C82", "C83", "C84", "C85", "C86", "C87", "C88", "C89", "C90", "C91", "C92", "C93", "C94", "C95", "C96", "A31", "A15", "A16", "A17", "A18", "A19", "B59", "Z87.01", "A81.2", "A02.1", "B58", "R64"]
    },
    'hiv': {
        '9': ["042"],
        '10': ["B20"]
    },
    'mst': {
        '9': ["196", "197", "198", "199.0"],
        '10': ["C77", "C78", "C79", "C80.0", "C80.2"]
    },
    'mal': {
        '9': ["14", "15", "16", "170", "171", "172", "174", "175", "176", "179", "18", "190", "191", "192", "193", "194", "195", "199.1", "200", "201", "202", "203", "204", "205", "206", "207", "208", "238.6"],
        '10': ["C0", "C1", "C2", "C30", "C31", "C32", "C33", "C34", "C37", "C38", "C39", "C40", "C41", "C43", "C45", "C46", "C47", "C48", "C49", "C50", "C51","C52","C53","C54","C55","C56","C57", "C58", "C60","C61","C62", "C63", "C76", "C80.1", "C81", "C82", "C83", "C84", "C85", "C88", "C9"]
    },
    'Obesity': {
        '9': ["278.0"],
        '10': ["E66"]
    },
    'WL': {
        '9': ['260','261','262','263','783.2','799.4'],
        '10': ['E40','E41','E42','E43','E44','E45','E46','R63.4','R64']
    },
    'Alcohol': {
        '9': ['265.2','291.1','291.2','291.3','291.5','291.8','291.9','303.0','303.9','305.0','357.5','425.5','535.3','571.0','571.1','5712','5713','980','V113'],
        '10': ['F10','E52','G62.1','I42.6','K29.2','K70.0','K70.3','K70.9','T51','Z50.2','Z71.4','Z72.1']
    },
    'Drug': {
        '9': ['292','304','305.2','305.3','305.4','305.5','305.6','305.7','305.8','305.9','V65.42'],
        '10': ['F11','F12','F13','F14','F15','F16','F18','F19','Z71.5','Z72.2']
    },
    'Psycho': {
        '9': ['293.8','295','296.04','296.14','296.44','296.54','297','298'],
        '10': ['F20','F22','F23','F24','F25','F28','F29','F30.2','F31.2','F31.5']
    },
    'Dep': {
        '9': ['296.2','296.3','296.5','300.4','309','311'],
        '10': ['F20.4','F31.3','F31.4','F31.5','F32','F33','F34.1','F41.2','F43.2']
    }
}

In [10]:
def merge_healthcare_data(db_path='healthcare.db'):
    conn = sqlite3.connect(db_path)
    
    charlson_conditions = []
    for condition, codes in CHARLSON_CONDITIONS.items():
        icd9_codes = "','".join(codes['9'])
        icd10_codes = "','".join(codes['10'])
        
        condition_sql = f"""
        MAX(CASE 
            WHEN (vocabulary_id = 'ICD9CM' AND diagnosis_code IN ('{icd9_codes}'))
            OR (vocabulary_id = 'ICD10CM' AND diagnosis_code IN ('{icd10_codes}'))
            THEN 1 ELSE 0 
        END) as {condition}"""
        charlson_conditions.append(condition_sql)
    
    charlson_sql = ",\n        ".join(charlson_conditions)
    
    query = f"""
    WITH screening_events AS (
        SELECT 
            p.patient_id,
            MIN(p.start_datetime) as first_screening_date,
            CASE WHEN COUNT(*) > 0 THEN 1 ELSE 0 END as event
        FROM procedure_table p
        WHERE p.procedure_code IN ('45378', '45380', '45384', '45385')
        GROUP BY p.patient_id
    ),
    last_visits AS (
        SELECT 
            e.patient_id,
            MAX(e.end_date) as last_visit_date
        FROM encounter e
        WHERE NOT EXISTS (
            SELECT 1 
            FROM procedure_table p 
            WHERE p.patient_id = e.patient_id 
            AND p.procedure_code IN ('45378', '45380', '45384', '45385')
        )
        GROUP BY e.patient_id
    )

    SELECT 
        d.patient_id,
        COALESCE(se.event, 0) as event,
        CASE 
            WHEN se.event = 1 THEN 
                ROUND((JULIANDAY(se.first_screening_date) - 
                      JULIANDAY(d.birth_date))/365.25 - 45, 2)
            ELSE 
                ROUND((JULIANDAY(lv.last_visit_date) - 
                      JULIANDAY(d.birth_date))/365.25 - 45, 2)
        END as T,
        d.sex,
        d.race,
        d.ethnicity,
        MIN(tt.travel_time_minutes) as min_travel_time,
        r.rucc_code as SDOH,
        d.education,
        d.income,
        {charlson_sql}
    FROM demographics d
    LEFT JOIN screening_events se ON d.patient_id = se.patient_id
    LEFT JOIN last_visits lv ON d.patient_id = lv.patient_id
    LEFT JOIN geolocation g ON d.patient_id = g.patient_id
    LEFT JOIN travel_time tt ON g.census_block = tt.census_block
    LEFT JOIN rucc r ON g.census_block = r.census_block
    LEFT JOIN diagnosis diag ON d.patient_id = diag.patient_id
    GROUP BY d.patient_id;
    """
    
    combined_df = pd.read_sql_query(query, conn)
    
    # Save both formats
    combined_df.to_csv('combined_data.csv', index=False)
    combined_df.to_parquet('combined_data.parquet')
    
    print("\nCombined Data Summary:")
    print(f"Total patients: {len(combined_df)}")
    print("\nScreening events distribution:")
    print(combined_df['event'].value_counts())
    
    # Charlson
    charlson_cols = CHARLSON_CONDITIONS.keys()
    print("\nCharlson Comorbidities Summary:")
    for col in charlson_cols:
        print(f"{col}: {combined_df[col].sum()} patients")
    
    print("\nTravel time statistics:")
    print(combined_df['min_travel_time'].describe())
    
    conn.close()
    return combined_df

In [11]:
if __name__ == "__main__":
   combined_data = merge_healthcare_data()
   print("\nSample of combined data:")
   print(combined_data.head())


Combined Data Summary:
Total patients: 100

Screening events distribution:
event
1    51
0    49
Name: count, dtype: int64

Charlson Comorbidities Summary:
mi: 5 patients
chf: 8 patients
pvd: 5 patients
cevd: 5 patients
dementia: 9 patients
copd: 9 patients
rheumd: 5 patients
pud: 5 patients
mld: 4 patients
msld: 3 patients
diab: 10 patients
dia_w_c: 5 patients
hp: 6 patients
mrend: 6 patients
srend: 8 patients
aids: 3 patients
hiv: 5 patients
mst: 4 patients
mal: 8 patients
Obesity: 5 patients
WL: 3 patients
Alcohol: 6 patients
Drug: 1 patients
Psycho: 4 patients
Dep: 5 patients

Travel time statistics:
count    100.000000
mean      52.481167
std       34.459595
min        3.300000
25%       16.754167
50%       57.091667
75%       79.662500
max      126.150000
Name: min_travel_time, dtype: float64

Sample of combined data:
  patient_id  event      T sex   race     ethnicity  min_travel_time  SDOH  \
0       P001      1  17.95   M  White  Non-Hispanic        69.600000     2   
1      